In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense
from tensorflow.keras.models import Model

# Prepare the data
df = [
    {'action': 2, 'point_differential': 0.0, 'ydstogo': 10.0, 'yards_to_goal': 75.0,
     'game_seconds_remaining': 3600.0, 'defense_power': 14.495989783170142,
     'offense_power': 33.511631990475884, 'yards_gained': 6.0, 'points_gained': 0.0},
    {'action': 3, 'point_differential': 0.0, 'ydstogo': 6.0, 'yards_to_goal': 61.0,
     'game_seconds_remaining': 3454.0, 'defense_power': 14.495989783170142,
     'offense_power': 33.511631990475884, 'yards_gained': 2.0, 'points_gained': 0.0}
]

actions = [0,1,2,3]

y = df[['yards_gained', 'points_gained']].values
X = df.drop(columns=['yards_gained', 'points_gained']).values

# Define the input layers
action_input = Input(shape=(1,))
metrics_input = Input(shape=(X.shape[1],))   # Shape correction

# Create the embedding layer for actions
embedding_dim = 8
action_embed = Embedding(input_dim=len(actions), output_dim=embedding_dim)(action_input)
action_flatten = tf.keras.layers.Flatten()(action_embed)

# Combine the input layers and action embedding
X_concat = tf.keras.layers.Concatenate()([action_flatten, metrics_input])
X = Dense(32, activation='relu')(X_concat)
X = Dense(32, activation='relu')(X)

# Task-specific layers for YARDS prediction
yards_layer = Dense(16, activation='relu')(X)
yards_output = Dense(1, name='yards_output')(yards_layer)

# Task-specific layers for POINTS prediction
points_layer = Dense(16, activation='relu')(X)
points_output = Dense(1, activation='sigmoid', name='points_output')(points_layer)

# Define the model with multiple outputs
model = Model(inputs=[action_input, metrics_input], outputs=[yards_output, points_output])

# Define loss functions and metrics for each output
model.compile(optimizer='adam',
              loss={'yards_output': 'mean_squared_error',
                    'points_output': 'binary_crossentropy'},
              metrics={'yards_output': 'mae',
                       'points_output': 'accuracy'})

# Train the model
model.fit(x=[X[:, 0], X[:, 1:]], y=[y[:, 0], y[:, 1]], epochs=10)



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the model architecture
class MultiTaskModel(nn.Module):
    def __init__(self, num_features):
        super(MultiTaskModel, self).__init__()
        self.shared_layer_1 = nn.Linear(num_features, 64)
        self.shared_layer_2 = nn.Linear(64, 32)
        self.yards_layer = nn.Linear(32, 16)
        self.yards_output = nn.Linear(16, 1)
        self.points_layer = nn.Linear(32, 16)
        self.points_output = nn.Linear(16, 1)

    def forward(self, x):
        shared = torch.relu(self.shared_layer_1(x))
        shared = torch.relu(self.shared_layer_2(shared))
        yards = torch.relu(self.yards_layer(shared))
        yards = self.yards_output(yards)
        points = torch.relu(self.points_layer(shared))
        points = self.points_output(points)
        return yards, points

# Create an instance of the model
model = MultiTaskModel(num_features)

# Define loss functions
yards_criterion = nn.MSELoss()
points_criterion = nn.BCELoss()

# Define optimizer
optimizer = optim.Adam(model.parameters())

# Train the model with multiple targets
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    yards_pred, points_pred = model(X_train)
    yards_loss = yards_criterion(yards_pred, y_train_yards)
    points_loss = points_criterion(points_pred, y_train_points)
    loss = yards_loss + points_loss
    loss.backward()
    optimizer.step()
